# Finger Spelling Recognition

In [1]:

import tensorflow as tf
import tensorflow_datasets as tfds
from tools.tqdm_notebook_callback import TQDMNotebookCallback

import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint


2023-06-24 18:17:38.070417: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-24 18:17:54.299741: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
print(tf.__version__)

2.10.0


In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  # return (tf.cast(image, tf.float32) / 255., tf.cast(label, tf.float32))
  return tf.cast(tf.expand_dims(image, -1), tf.float32) / 255., label

In [ ]:
ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.batch(512)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(512)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [ ]:
wandb.init(
    project="mnist-playground",
    name="non-distributed",
    config={
        "learning_rate": 0.001,
        # "epochs": 10,
        # "batch_size": 128
    }
)

config = wandb.config

In [ ]:
# wandb.finish()

In [ ]:
# num_classes = 10
input_shape = (28, 28, 1)

# mirrored_strategy = tf.distribute.MirroredStrategy()

# with mirrored_strategy.scope():

model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=input_shape),
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation="softmax"),
])



    # model = tf.keras.Sequential([
    #     tf.keras.layers.Flatten(input_shape=input_shape),
    #     tf.keras.layers.Dense(128, activation='relu'),
    #     tf.keras.layers.Dense(10, activation="softmax"),
    # ])

model.compile(
    # optimizer="adam",
    # optimizer=tf.keras.optimizers.Adam(config.learning_rate),
    optimizer=tf.keras.optimizers.legacy.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    # metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
    metrics=["accuracy", "mse"]
)

# model.summary()



In [ ]:
tf.get_logger().setLevel('ERROR')

model.fit(
    # x=x_train,
    # y=y_train,
    ds_train,
    # batch_size=config.batch_size,
    # epochs=config.epochs,
    # batch_size=128,
    epochs=10,
    # validation_split=0.1,
    validation_data=ds_test,
    verbose=0,
    callbacks=[
        # WandbMetricsLogger(log_freq=5),
        TQDMNotebookCallback(),
    ]
)

# wandb.finish()

In [ ]:
# wandb.init(
#     project="mnist-playground",
#     name="distributed",
#     config={
#         "learning_rate": 0.001,
#         "epochs": 10,
#         "batch_size": 128
#     }
# )

# config = wandb.config

In [ ]:
# mirrored_strategy = tf.distribute.MirroredStrategy()

# with mirrored_strategy.scope():
#   distributed_model = tf.keras.models.Sequential([
#     tf.keras.layers.Flatten(input_shape=(28, 28)),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dense(10)
#   ])

# distributed_model.compile(
#     optimizer=tf.keras.optimizers.Adam(config.learning_rate),
#     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#     metrics=[tf.keras.metrics.SparseCategoricalAccuracy(),
#              tf.keras.metrics.MeanSquaredError(),
#              tf.keras.metrics.MeanAbsoluteError()
#             ],
# )

# distributed_model.fit(
#     ds_train,
#     batch_size=config.batch_size,
#     epochs=config.epochs,
#     validation_data=ds_test,
#     verbose=0,
#     callbacks=[
#         WandbMetricsLogger(log_freq=5),
#         TQDMNotebookCallback()
#     ]
# )

# wandb.finish()